## 2. Segy text header

Textual Header - заголовок размера 3200 байт, в котором содержится текстовое описание SEG-Y файла. См. SEG Technical Standards Committee SEG-Y revision 2.0 Data Exchange format (https://seg.org/Portals/0/SEG/News%20and%20Resources/Technical%20Standards/seg_y_rev2_0-mar2017.pdf)

Рассмотрим функционал библиотеки для работы с текстовым заголовком.

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [2]:
import seismo_reader as sr

### 2.1 Описание класса

Текстовый заголовок представлен классом sr.segy_text_header. Описание методов класса можно получить по команде help:

In [3]:
help(sr.segy_text_header)

Help on class segy_text_header in module seismo_reader:

class segy_text_header(abstract_header)
 |  Method resolution order:
 |      segy_text_header
 |      abstract_header
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: seismo_reader.segy_text_header) -> None
 |      
 |      2. __init__(self: seismo_reader.segy_text_header, arg0: List[int], arg1: bool) -> None
 |  
 |  raw_data(...)
 |      raw_data(self: seismo_reader.segy_text_header) -> List[int]
 |  
 |  set_object_name(...)
 |      set_object_name(self: seismo_reader.segy_text_header, arg0: str) -> None
 |  
 |  set_product_name(...)
 |      set_product_name(self: seismo_reader.segy_text_header, arg0: str) -> None
 |  
 |  set_seismic_type(...)
 |      set_seismic_type(self: seismo_reader.segy_text_header, arg0: str) -> None
 |  
 |  --------------------------

Так как общепринятый вормат текстового заголовка - это 40 строк по 80 символов, а каждая строка начинается с 'CN', где N - номер строки, класс реализован в виде хранилища ключ-значение, где ключами являются строки вида 'CN', а значениями - остальные символы строк.

Класс поддерживает преобразование в/из __dict__.

Дополнительно представлены функции __get/set__, позволяющие манипулировать отдельными полями

Доступен как пустой конструктор, так и конструктор из словаря. Есть возможность получить "сырые" 3200 байт как в формате EBDIC, так и в ASCII.

### 2.2 Чтение и просмотр текстового заголовка из SEG-Y файла

Рассмотрим пример чтения бинарного заголовка из SEG-Y файла, опробуем функционал класса.

In [9]:
config = sr.reader_config()
config.filename = '2D.segy'
reader = sr.segy_reader(config)

Получить текстовый заголовок из открытого SEG-Y файла можно методом segy_reader.text_header().

In [12]:
th = reader.text_header()

Преобразуем бинарный заголовок в словарь и посмотрим на его содержимое.

In [13]:
th.to_dict()

{'C 1': 'SEGY OUTPUT FROM Petrel 2013.1 (64-bit) Tuesday, November 26 2013 09:30:10  ',
 'C 2': 'Name: 30389-07_0  PhaseShift@ Type: 2D seismic                              ',
 'C 3': '                                                                            ',
 'C 4': 'First CDP: 0.000000 Last CDP: 326.000000                                    ',
 'C 5': 'First SP:  0.000000 Last SP:  0.000000                                      ',
 'C 6': 'CRS: <undef>                                                                ',
 'C 7': 'X min: 123207.00 max: 132229.00 delta: 9022.00                              ',
 'C 8': 'Y min: 1119097.00 max: 1132618.00 delta: 13521.00                           ',
 'C 9': 'Time min: -3556.00 max: 46.00 delta: 3602.00                                ',
 'C10': 'Lat min: - max: - delta: -                                                  ',
 'C11': 'Long min: - max: - delta: -                                                 ',
 'C12': 'Trace min: -3555.00 max

Значение по полю можно получить не только из словаря, но и методом get:

In [15]:
th.get('C19')

'Sample interval             : bytes 17-18                                   '

Отдельно стоит рассмотреть поля 'C 1' - 'C 9', в которых номер отделен от буквы C. Допускается получение таких полей как по ключам 'C 1'...'C 9', так и по ключам 'C1'...'C9':

In [16]:
th.get('C 1')

'SEGY OUTPUT FROM Petrel 2013.1 (64-bit) Tuesday, November 26 2013 09:30:10  '

In [17]:
th.get('C1')

'SEGY OUTPUT FROM Petrel 2013.1 (64-bit) Tuesday, November 26 2013 09:30:10  '

Обработка ошибок класса segy_text_header осуществляется посредством исключений.

Попытаемся получить несуществующую строку текстового заголовка.

In [18]:
th.get('C41')

ValueError: segy_text_header.cpp:65: invalid field name

Получаем исключение __ValueError__ с описанием _invalid field name_.

Так же можно посмотреть (или манипулировать) с "сырым" заголовком через метод __raw_data__:

In [25]:
a = np.array(th.raw_data(), dtype=np.int8)

Размер такого массива, как и ожидалось, 3200 байт:

In [26]:
a.shape

(3200,)

### 2.3 Создание и редактирование бинарного заголовка

Для изменения содержимого текстового заголовка предоставляются методы __set/from_dict__ и конструктор __init__(self, dict).

В качестве ключа в метод __set__ подается строка вида __'C N'__, в качестве значения подается строка из не более чем 76 символов. Аналогично для редактирования текстового заголовка через словарь.

Изменять текстовый заголовок, сопряженный со своим файлом, запрещено. Действительно, попробовав изменить полученный из файла заголовок, получаем исключение (object is read only):

In [29]:
th.from_dict({ 'C 1': 'ABCD' })

RuntimeError: segy_text_header.cpp:74: object is read only

In [30]:
th.set('C 1', 'ABCD')

RuntimeError: segy_text_header.cpp:74: object is read only

Создадим пустой текстовый заголовок.

In [44]:
th_new = sr.segy_text_header()

Выведем его поля в виде словаря.

In [45]:
d = th_new.to_dict()

for k, v in d.items():
    print(f'{k}: {v}')

C 1: SEG-Y OUTPUT FROM %ProductName, %DateTime
C 2: NAME: %name	Type: %type
C 3: 
C 4: Additional text headers count:	%add_text_headers_count
C 5: 
C 6: AREA INFO
C 7: First CDP:	%first_cdp,	Last CDP:	%last_cdp
C 8: X min:		%x_min,	X max:		%x_max,	Delta:	%x_delta
C 9: Y min:		%y_min,	Y max:		%y_max,	Delta:	%y_delta
C10: Time min:	%time_min,	Time max:	%time_max,	Delta:	%time_delta
C11: Lat min:	%lat_min,	Lat max:	%lat_max,	Delta:	%lat_delta
C12: Long min:	%long_min,	Long max:	%long_max,	Delta:	%long_delta
C13: Trace min:	%trace_min,	Trace max:	%trace_max,	Delta:	%trace_delta
C14: 
C15: BINARY HEADER INFO
C16: Location:						%bin_header_loc
C17: Sample interval:				%bin_header_sample_interval_loc
C18: Number of samples per trace:	%bin_header_samples_count_loc
C19: Trace data format:				%bin_header_data_format_loc
C20: 
C21: TRACE HEADER INFO
C22: Location:						%trace_header_loc
C23: Inline number:					%trace_header_iline_loc
C24: Crossline number:				%trace_header_crossline_loc
C25: CDP

Видим, что у пустого текстового заголовка уже есть некоторый шаблон, в котором присутствуют паттерн для автозамены. Например, вместо __%ProductName__ можно подставить название программного продукта из которого выгружается SEG-Y файл.

Самостоятельная автозамена полей нежелательна, но возможна. Например, можно заменить паттерн __%DateTime__ на дату активного написания библиотеки seismo_reader: __24.07.2020 02:00__.

In [46]:
new_C1_str = th_new.get('C 1').replace('%DateTime', '24.07.2020 02:00')

In [47]:
th_new.set('C 1', new_C1_str)

Снова выводим на экран словарь текстового заголовка и видим, что паттерн даты заменился на установленное значение:

In [48]:
d = th_new.to_dict()

for k, v in d.items():
    print(f'{k}: {v}')

C 1: SEG-Y OUTPUT FROM %ProductName, 24.07.2020 02:00
C 2: NAME: %name	Type: %type
C 3: 
C 4: Additional text headers count:	%add_text_headers_count
C 5: 
C 6: AREA INFO
C 7: First CDP:	%first_cdp,	Last CDP:	%last_cdp
C 8: X min:		%x_min,	X max:		%x_max,	Delta:	%x_delta
C 9: Y min:		%y_min,	Y max:		%y_max,	Delta:	%y_delta
C10: Time min:	%time_min,	Time max:	%time_max,	Delta:	%time_delta
C11: Lat min:	%lat_min,	Lat max:	%lat_max,	Delta:	%lat_delta
C12: Long min:	%long_min,	Long max:	%long_max,	Delta:	%long_delta
C13: Trace min:	%trace_min,	Trace max:	%trace_max,	Delta:	%trace_delta
C14: 
C15: BINARY HEADER INFO
C16: Location:						%bin_header_loc
C17: Sample interval:				%bin_header_sample_interval_loc
C18: Number of samples per trace:	%bin_header_samples_count_loc
C19: Trace data format:				%bin_header_data_format_loc
C20: 
C21: TRACE HEADER INFO
C22: Location:						%trace_header_loc
C23: Inline number:					%trace_header_iline_loc
C24: Crossline number:				%trace_header_crossline_loc
C

Таким способом можно не только заменинить некоторые (или все) паттерны, но и полностью изменить текстовый заголовок на свой вкус.

Однако, по-хорошему такая подстановка является костылем. Класс segy_text_header предоставляет лишь несколько методов для редактирования паттернов, которые приведены ниже:
>__set_product_name__<br>
>__set_object_name__<br>
>__set_seismic_type__<br>

Все остальные паттерны автоматически подставятся при записи трасс в файл, а огромные отступы примут красивый ровный вид (в теории).

Попробуем метод __set_product_name__. Изначальная строка __C 1__ имеет вид:

In [49]:
th_new.get('C 1')

'SEG-Y OUTPUT FROM %ProductName, 24.07.2020 02:00'

Установим новое значение для __%ProductName__.

In [50]:
th_new.set_product_name('RN-GEOSIM 1.0.4.100.200.300alpha.beta.gama')

Получаем:

In [51]:
th_new.get('C 1')

'SEG-Y OUTPUT FROM RN-GEOSIM 1.0.4.100.200.300alpha.beta.gama, 24.07.2020 02:00'

При попытке установить строку более 76 символов получаем исключение:

In [53]:
th_new.set('C 1', '1234567891012345678920123456789301234567894012345678950123456789601234567897012345678980')

ValueError: segy_text_header.cpp:85: invalid text header field size (use string with length not greater than 76 characters)

И поле остается необновленным:

In [55]:
th_new.get('C 1')

'SEG-Y OUTPUT FROM RN-GEOSIM 1.0.4.100.200.300alpha.beta.gama, 24.07.2020 02:00'